In [ ]:
import torch
from torch import functional as F
from typing import Callable, Tuple
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
import matplotlib
rc('animation', html='jshtml')

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [ ]:
class GameRule(torch.nn.Module):
    def __init__(self, zero_kernel, one_kernel, zero_bias, one_bias, activation_zero, activation_one):
        super().__init__()
        self.zero_kernel = zero_kernel.unsqueeze(0).unsqueeze(0).to(device)
        self.one_kernel = one_kernel.unsqueeze(0).unsqueeze(0).to(device)
        self.zero_bias = torch.tensor([zero_bias]).to(device)
        self.one_bias = torch.tensor([one_bias]).to(device)
        self.activation_zero = activation_zero
        self.activation_one = activation_one

    def forward(self, input):
        conv0 = torch.conv2d(input=input, weight=self.zero_kernel, bias=self.zero_bias, padding='same', stride=1)
        conv1 = torch.conv2d(input=input, weight=self.one_kernel, bias=self.one_bias, padding='same', stride=1)
        return (input > 0.5) * self.activation_one(conv1) + (input < 0.5) * self.activation_zero(conv0)

In [ ]:
def get_baseline_rule() -> GameRule:
    def baseline_activation1(board: torch.Tensor) -> torch.Tensor:
        mask = (board >= 2) * (board <= 3)
        return mask.to(torch.float32)
    def baseline_activation0(board: torch.Tensor) -> torch.Tensor:
        mask = (board - 3).abs() < 1e-5
        return mask.to(torch.float32)
    kernel = torch.tensor([[1, 1, 1], [1, 0, 1], [1, 1, 1]]).to(torch.float32)
    return GameRule(kernel, kernel, 0., 0., baseline_activation0, baseline_activation1)

In [ ]:
from ipywidgets import HTML


def simulate(initial_state: torch.Tensor,
             rule: GameRule,
             interval: int=500,
             transitions: int = 20,
             visualize: bool = True) -> torch.Tensor:
    """
    Animates first n states of CGoL given the initial state

    :param initial_state: the initial state of the board
    :param rule: the update rule
    :param interval: Delay between frames in milliseconds.
    :param transitions: Number of transitions to simulate.
    :param visualize: If true, visualize the transitions
    :return final state of the board
    """

    current_state = initial_state.unsqueeze(0).unsqueeze(0).to(device)

    if visualize:
        fig, ax = plt.subplots()
        img = ax.imshow(initial_state.numpy(), cmap='jet', vmin=0, vmax=1)  # Set color map and value range
        plt.colorbar(img, ax=ax)

        def update(frame):
            nonlocal current_state
            current_state = rule(current_state)
            img.set_array(current_state.squeeze().numpy())
            return [img]

        ani = animation.FuncAnimation(fig, update, frames=transitions, interval=interval, blit=False)
        return ani

    else:
        for _ in range(transitions):
            rule(current_state)

    return current_state


In [ ]:
#%matplotlib notebook
simulate(torch.randint(0, 2, (100, 100), dtype=torch.float), get_baseline_rule(), transitions=1000, interval=200)

# Continuous activation
Now we wil replace step activation functions with normal distribution. I will adjust new activations to keep the integral of activation functions at the same level. (When integrating over reals, which ofc is not 100% proper)

In [ ]:
def get_continuous_rule() -> GameRule:
    def continuous_activation1(board: torch.Tensor) -> torch.Tensor:
        dist = torch.distributions.normal.Normal(2, 1.25)
        return torch.exp(dist.log_prob(torch.Tensor(board))) * 2
    def continuous_activation0(board: torch.Tensor) -> torch.Tensor:
        dist = torch.distributions.normal.Normal(3, 0.25)
        return torch.exp(dist.log_prob(torch.Tensor(board))) * 2
    kernel = torch.tensor([[1, 1, 1], [1, 0, 1], [1, 1, 1]]).to(torch.float32)
    return GameRule(kernel, kernel, 0., 0., continuous_activation0, continuous_activation1)

In [ ]:
simulate(torch.rand((50, 50), dtype=torch.float), get_continuous_rule(), transitions=100, interval=200)

I have realised that the transition rules defined above behaves strangely because I let the cell value exceed `1`. In the following section I will fix this. I will leave the improper transition rules for educational purposes.

In [ ]:
EPSILON = 1e-6
def get_continuous_rule_fixed(alpha = 2.5) -> GameRule:
    def continuous_activation1(board: torch.Tensor) -> torch.Tensor:
        board -= 2.5
        board *= alpha
        return (torch.sin(board) / abs(board) + EPSILON) ** 2
    def continuous_activation0(board: torch.Tensor) -> torch.Tensor:
        board -= 3
        board *= alpha
        return (torch.sin(board) / abs(board) + EPSILON) ** 2
    kernel = torch.tensor([[1, 1, 1], [1, 0, 1], [1, 1, 1]]).to(torch.float32)
    return GameRule(kernel, kernel, 0., 0., continuous_activation0, continuous_activation1)

Given this formula, I will try to find value of alpha such that expected cell value after first iteration will still be 0.5.

In [ ]:
simulate(torch.rand((100, 100), dtype=torch.float), get_continuous_rule_fixed(alpha=2.2), transitions=500, interval=200)

Lets try asymmetrical discrete rule:

In [ ]:
def get_asym_discrete_rule() -> GameRule:
    def baseline_activation1(board: torch.Tensor) -> torch.Tensor:
        mask = (board >= 2) * (board <= 3)
        return mask.to(torch.float32)
    def baseline_activation0(board: torch.Tensor) -> torch.Tensor:
        mask = (board - 3).abs() < 1e-5
        return mask.to(torch.float32)
    kernel = torch.tensor([[1, 1, 1], [2, 0, 1], [1, 1, 1]]).to(torch.float32)
    return GameRule(kernel, kernel, 0., 0., baseline_activation0, baseline_activation1)

In [ ]:
simulate(torch.randint(0, 2, (100, 100), dtype=torch.float), get_asym_discrete_rule(), transitions=1000, interval=200)

The rule above creates a lot of simple gliders.

# Continuous asymmetric rule

In [ ]:
def get_continuous_asymmetric_rule(alpha = 2.5) -> GameRule:
    def continuous_activation1(board: torch.Tensor) -> torch.Tensor:
        board -= 2.5
        board *= alpha
        return (torch.sin(board) / abs(board) + EPSILON) ** 2
    def continuous_activation0(board: torch.Tensor) -> torch.Tensor:
        board -= 3
        board *= alpha
        return (torch.sin(board) / abs(board) + EPSILON) ** 2
    kernel = torch.tensor([[1.3, 0.8, 0.9], [1, 0, 1], [1, 1, 1]]).to(torch.float32)
    return GameRule(kernel, kernel, 0., 0., continuous_activation0, continuous_activation1)

In [ ]:
simulate(torch.rand((100, 100), dtype=torch.float), get_continuous_asymmetric_rule(alpha=2.2), transitions=200, interval=200)

In [ ]:
def get_monoconv_rule():
    def activation(board: torch.Tensor) -> torch.Tensor:
        return torch.nn.functional.sigmoid(board)
    kernel = torch.tensor([[.15, .1, .1], [0.2, 0.7, 0.1], [0.05, 0.12, 0.1]]).to(torch.float32) / 2.
    return GameRule(kernel, kernel, 0., 0., activation, activation)

In [ ]:
simulate(torch.rand((100, 100), dtype=torch.float), get_monoconv_rule(), transitions=200, interval=200)

## Sine rule

In [ ]:
def get_sine_rule(delta: float = 0.5) -> GameRule:
    def activation(board: torch.Tensor) -> torch.Tensor:
        return (torch.sin(board) + 1) / 2
    kernel = torch.tensor([[1., 1., 1.], [1., 1., 1.], [1., 1., 1.]]).to(torch.float32) * delta
    return GameRule(kernel, kernel, 0., 0., activation, activation)

def get_cosine_rule(delta: float = 0.5) -> GameRule:
    def activation(board: torch.Tensor) -> torch.Tensor:
        return (torch.cos(board) + 1) / 2
    kernel = torch.tensor([[1., 1., 1.], [1., 1., 1.], [1., 1., 1.]]).to(torch.float32) * delta
    return GameRule(kernel, kernel, 0., 0., activation, activation)

In [ ]:
simulate(torch.rand((100, 100), dtype=torch.float) / 1000., get_sine_rule(delta=0.174), transitions=100, interval=200)

In [ ]:
simulate(torch.rand((100, 100), dtype=torch.float), get_cosine_rule(), transitions=200, interval=200)

In [ ]:
def rule_converges(rule: GameRule, trials: int, steps: int = 100, board_size: Tuple[int, int] = (100, 100), look_back: int = 1) -> float:
    attempts_converged = 0
    boards = torch.rand((trials, *board_size))
    last_steps = []
    for i in range(steps):
        if i >= steps - look_back:
            last_steps.append(boards.clone())
        boards = rule(boards)

    for i, b in enumerate(boards):
        for copy in last_steps:
            if torch.isclose(b, copy[i]):
                attempts_converged += 1
                break

    return attempts_converged / trials
